# PREAMBULE

## Synopsis

Purpose 
A partir des données de transit de bateau d'un port, générer un set de paramètres pour le PAS modelling. Non pas un set final, mais une première
trame. 
Principe 
Basé sur le cas des données de l'IH de GPMB: - prendre les infos exploitable - sortir pr chaque categorie de cargaison : - le segment - la liste des
docks observés - ajouter les info nécessaire pour générer Rules (on présupose une SC par tuple (direction, segment)) 
Use Example 
Current state 
Embryonnaire : simplement la restructuration des infos sous un df pertinent. 
To Do 
[] Générer les différents json de paramètres. [] N'enregistrer un handling que s'il respecte le json-schema (attention, on parle ici d'une sous
structure précisément pr un handling, pas pour l'input CHR entier) ==> subdiviser le json-schema de CHR et l'appeler pr le test [] A la cloture,
vérifier que l'input est valide vis à vis des schémas : - par item - au global 
Change Lo

*PURPOSE*

Ceci est un fork du builder de set de paramètres (en début de dév). Il est ici détourné pr vérifier les sorties du convertiseur.

*TODO*


## Librairies

In [1]:
# GENERAL

import pandas as pd
import numpy as np

In [2]:
# GESTION JSON

#from pandas.io.json import json_normalize #package for flattening json in pandas df <<Périmé en pandas v1
import pprint 
import json
import jsonschema

In [3]:
# AFFICHAGE NOTEBOOK

from IPython.core.interactiveshell import InteractiveShell # Mettre en output toutes les sortie d'une cellule
InteractiveShell.ast_node_interactivity = "all" #"last_expr"

In [4]:
# Mise en forme du notebook 

# Nb de colonnes/ligne à afficher
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# Largeur des cellules #Uniquement pr Jup Notbook ?
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Mettre en output toutes les sortie d'une cellule
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" #"last_expr"

# Avoir une forme de débugger
from IPython.core.debugger import Tracer # Plancer Tracer()() dans la fonction permet d'avoir un pas à pas, "n(ext) line and run this one, c(ontinue) running until next breakpoint, q(uit) the debugger"

## Fonctions

In [5]:
def leafExplorer(df, listeBranches):
    '''
    A partir d'un df contenant des colonnes dont les cellules sont des dic (et NON des array de dic), 
    extrait les dictionnaires ou les array et passe les clé en nom de colonne, puis les valeurs en "cellule".
    NB : on a pr nom de colonne uniquement la clé du dic
    '''
   
    for branche in listeBranches :
        try :
            branche in df.columns
        except :
            print(branche, " n'est pas un nom de colonne présent dans le df donné !")
            
        try :
            isinstance(df[branche][0], dict)
        except:
            print("Le contenu de ", branche, " n'est pas toujours un dict !")
        finally:
            df = df.drop(branche, 1).assign(**pd.DataFrame(df[branche].values.tolist()))
    return df;

In [6]:
def jprint (json_like_item, structure = 'dict'):
    '''
    Affiche de manière plus lisible le json (instance sous forme de string)
    '''
    if structure == 'dict':
        print(
            json.dumps(
                json.load(
                    json_like_item), 
                indent=4, 
                sort_keys=True
            )
        )
    elif structure == 'json':
        print(
            json.dumps(
                json.loads(
                    json_like_item), 
                indent=4, 
                sort_keys=True
            )
        )

In [7]:
# Export du json

def exportJson(item, full_path):
    with open(full_path, "w") as file:
            file.write(
                json.dumps(
                    item, 
                    indent=4, 
                    ensure_ascii=False
                )
            )

## Tampon bouts de code

# STOPOVERS IH

In [9]:
#CHARGEMENT


with open("../../inputs/GPMB/IH_2018.json") as json_file:
    ih_df = json.load(json_file)

ih_df = pd.json_normalize(
    ih_df[0]["records"], #On filtre sur 0 car on a un seul terminal présent
    sep="__"
)

ih_df.columns = ['timestamp', 'departure_handling_dock', 'loading_cargo_type', 'IMO', 'loading_berth', 'source', 'loading_cargo_fiscal_type', 'journeyid', 'loading_dangerous', 'unloading_tonnage', 'arrival_handling_dock', 'pixel_etd', 'unloading_cargo_fiscal_type', 'loading_tonnage', 'unloading_cargo_type', 'unloading_agent', 'unloading_berth', 'name', 'unloading_dangerous', 'location__lon', 'location__lat', 'dataProvider', 'operation', 'loading_agent']


#ih_df.shape
ih_df

KeyError: 'records'

In [220]:
# EXTRACTION


nomsColonnes = ["handling_type", "content_type", "handling_dock", "content_amount", "handling_direction"]

load = ih_df.loc[:, ["loading_cargo_fiscal_type", "loading_cargo_type", "loading_berth", "loading_tonnage", "operation"]]
load.columns = nomsColonnes

unload = ih_df.loc[:, ["unloading_cargo_fiscal_type", "unloading_cargo_type", "unloading_berth", "unloading_tonnage", "operation"]]
unload.columns = nomsColonnes
    
data = pd.concat([load, unload])

data.replace({"":np.nan, 0:np.nan}, inplace=True)
data.dropna(subset=["handling_type", "content_type", "handling_dock", "content_amount", "handling_direction"], inplace=True)

col= ["handling_dock", "content_amount"]
data[col] = data[col].astype('int32')
#data.astype({}, copy=False)
data["handling_dock"] = data["handling_dock"].astype('str') #On conserve la première conversion en int pr ne pas avoir les traces du passage par float

# Pr la handling_direction, on ne peut pas selectionner le cas le plus fréquent (comme pr handling_type), mais on doit forcer
data.loc[data.content_type.str.startswith("E"), "handling_direction"] = "loading"
data.loc[data.content_type.str.startswith("I"), "handling_direction"] = "unloading"


data
len(data.groupby(["handling_type", "handling_direction"])["content_amount"].count())
data.groupby(["handling_type", "handling_direction"])["content_amount"].count()

,handling_type,content_type,handling_dock,content_amount,handling_direction
3,LIQ.V,E.HUILE TOURNESOL,436,2424,loading
5,CEREAL,E.MAIS VRAC,449,4522,loading
9,P.C.,E.CONTENEURS,432,1217,loading
10,SOL.V,E.FERRAIL.REFONT.,414,4666,loading
11,CEREAL,E.MAIS VRAC,449,3810,loading
...,...,...,...,...,...
436,GAZ.L,I.BUTADIENE,436,2100,unloading
437,LIQ.V,I.HUILE COLZA,436,2997,unloading
439,SOL.V,I.SEL,433,3122,unloading
440,LIQ.V,I.EMHV - EMAG - FAME,436,1998,unloading


11

handling_type  handling_direction
AUTRES         loading                 2
               unloading              23
CEREAL         loading               102
GAZ.L          loading                 1
               unloading              27
LIQ.V          loading                49
               unloading              68
P.C.           loading                67
               unloading              66
SOL.V          loading                33
               unloading              72
Name: content_amount, dtype: int64

In [221]:
# RESHAPING

data_core = pd.DataFrame(
    data.groupby(["content_type", "handling_direction"])["handling_type"].agg(lambda x:x.value_counts().index[0]) #On prend uniquement la valeur la plus fréquence (possible égalité)
)

data_core.insert(1, "handling_dock", 
                 data.groupby(["content_type", "handling_direction"])["handling_dock"].apply(lambda x: pd.unique(x).tolist())) #On passe en valeur la liste de tous les handling_docks ayant correspondus

data_core.reset_index(inplace= True)

data_core["supplychain"]= data_core["handling_direction"] + '_' + data_core["handling_type"]


data_core.shape
data_core.head()

(48, 5)

,content_type,handling_direction,handling_type,handling_dock,supplychain
0,E.BLE VRAC,loading,CEREAL,[449],loading_CEREAL
1,E.CER.+OLEAG. MIXTES,loading,CEREAL,"[417, 449]",loading_CEREAL
2,E.CEREALES MIXTES VRAC,loading,CEREAL,"[449, 417]",loading_CEREAL
3,E.COLIS LOURDS,loading,AUTRES,"[413, 417]",loading_AUTRES
4,E.COLZA VRAC,loading,CEREAL,[449],loading_CEREAL


# RULES

## Content_Type

In [222]:
def content_type_jsonification(ligne):
    item_dict = {
        "content_type_ID": ligne.content_type,
        "label":"Some label",
        "comment": "Some comment",
        "handling_type": ligne.handling_type, #parler de catégorie, famille, parent ?
        "unit": "Tons",
        "possible_SC_assignation_list": [
            {
                "supplychain": ligne.supplychain,
                "restrictions": {
                    "direction":ligne.handling_direction,
                    "dock": ligne.handling_dock,
                    "amount_min": 1000,
                    "amount_max": 50000
                }
            }
        ]
    }
    return item_dict

RULES_content_type_list = {
    "content_type_list": list(
        data_core.apply(
            content_type_jsonification, 
            axis= 1,
            result_type= 'reduce'
        )
    )
}

## Cloture RULES

In [223]:
RULES = {
    "content_type_list": list(
        data_core.apply(
            content_type_jsonification, 
            axis= 1,
            result_type= 'reduce'
        )
    ),
    "shiftworks":[
        {
            "ID":"typical",
            "value":[
                {"Monday":["8>12","14>17"]},
                {"Tuesday":["8>12","14>17"]},
                {"Wednesday":["8>12","14>17"]},
                {"Thursday":["8>12","14>17"]},
                {"Friday":["8>12","14>17"]},
                {"Saturday":[]},
                {"Sunday":[]}
            ]
        }
    ],
    "handling_priority": {}
}

In [224]:
exportJson(RULES, "../../inputs/GPMB/RULES.json")

# SUPPYCHAINS

In [ ]:
# Préparation des données 

df = data_core
df.columns = ["cargoes_content_type", "handling_directions_nature", "handling_type", "handling_dock_ID", "ID"]

supChain_df = pd.DataFrame(
    data= df.groupby(["ID", "handling_directions_nature"])["cargoes_content_type"].apply(lambda x: pd.unique(x).tolist()),
    columns=["cargoes_content_type"]
)

supChain_df.insert(1, "areas_ID", 
    df.groupby(["ID", "handling_directions_nature"])["handling_dock_ID"].apply(lambda x: list(set(x.sum())))
)

supChain_df.reset_index(inplace= True)


supChain_df

In [ ]:
# Convertion en json

supChain_Json = json.loads(supChain_df.to_json(orient='records', indent=4))

for item in supChain_Json :
    item["ID"]= item["ID"]
    item["label"]= "Some label"
    item["comment"]= "Some comment"
    item["compatibility"]= {
        "cargoes_content_type": item["cargoes_content_type"],
        "handling_directions_nature": [item["handling_directions_nature"]],
        "areas_ID": item["areas_ID"],
        "shiftworks_ID": ["typical"]
    }
    item["steps_list"]= [
        {
            "ID": "Step1",
            "label": "step's name",
            "comment": "step's comment",
            "content_type": "Initialization",
            "scheduling": {
                "start": {
                    "nature": "delay",
                    "value": 0
                },
                "duration": {
                    "nature": "delay",
                    "value": 20
                }
            },
            "work": {
                "machines": [
                    "Machine1"
                ],
                "nature": "parallel",
                "distance": "on_spot"
            }
        },
        {
            "ID": "Step2",
            "label": "step's name",
            "comment": "step's comment",
            "content_type": "Processing",
            "scheduling": {
                "start": {
                    "nature": "after_any",
                    "value": ["Step1"]
                },
                "duration": {
                    "nature": "cargo_%",
                    "value": 100
                }
            },
            "work": {
                "machines": [
                    "Machine2, Machine3"
                ],
                "nature": "parallel",
                "distance": "on_spot"
            }
        }
    ]


pprint.PrettyPrinter(indent=4).pprint(supChain_Json[0])

In [ ]:
#lister l'ensemble des machines référencées dans le json
supChain_Json[]["ID"]

# RESSOURCE

In [ ]:
#load json object
with open('./RESSOURCES_demo.json') as json_file:
    ressources = json.load(json_file)

ressources = json_normalize(ressources['machines'])


ressources.head()

In [ ]:


others_list = []

In [ ]:
# AREAS

areas_list = []

for handling_dock in data.handling_dock.unique() :
    areas_list.append(
        {
            "ID": handling_dock,
            "label": "some label",
            "comment": "some comment",
            "content_type": "handling_dock",
            "owner": "the owner"
            "terminal": "the terminal"
            
        }
    )

In [ ]:
# MACHINES

machines_list = []

for machine in 

In [ ]:
ressource = {
    "areas_list":areas_list,
    "machines_list":machines_list,
    "others_list":others_list
}


print(json.dumps(ressource, indent=4))

# NEW PORT SET

## Content-types extraction

In [17]:
#CREATION DF

with open("/media/erwan/DATA1/CODE/1_CODE_PIXEL/1_PAS_MODELLING/tools/demo_inputs/GPMB_2018/Handlings.json") as json_file:
    vesselCalls_df = json.load(json_file)
    
vesselCalls_df = pd.json_normalize(vesselCalls_df)

In [220]:
# EXTRACTION


nomsColonnes = ["handling_type", "content_type", "handling_dock", "content_amount", "handling_direction"]

load = ih_df.loc[:, ["loading_cargo_fiscal_type", "loading_cargo_type", "loading_berth", "loading_tonnage", "operation"]]
load.columns = nomsColonnes

unload = ih_df.loc[:, ["unloading_cargo_fiscal_type", "unloading_cargo_type", "unloading_berth", "unloading_tonnage", "operation"]]
unload.columns = nomsColonnes
    
data = pd.concat([load, unload])

data.replace({"":np.nan, 0:np.nan}, inplace=True)
data.dropna(subset=["handling_type", "content_type", "handling_dock", "content_amount", "handling_direction"], inplace=True)

col= ["handling_dock", "content_amount"]
data[col] = data[col].astype('int32')
#data.astype({}, copy=False)
data["handling_dock"] = data["handling_dock"].astype('str') #On conserve la première conversion en int pr ne pas avoir les traces du passage par float

# Pr la handling_direction, on ne peut pas selectionner le cas le plus fréquent (comme pr handling_type), mais on doit forcer
data.loc[data.content_type.str.startswith("E"), "handling_direction"] = "loading"
data.loc[data.content_type.str.startswith("I"), "handling_direction"] = "unloading"


data
len(data.groupby(["handling_type", "handling_direction"])["content_amount"].count())
data.groupby(["handling_type", "handling_direction"])["content_amount"].count()

,handling_type,content_type,handling_dock,content_amount,handling_direction
3,LIQ.V,E.HUILE TOURNESOL,436,2424,loading
5,CEREAL,E.MAIS VRAC,449,4522,loading
9,P.C.,E.CONTENEURS,432,1217,loading
10,SOL.V,E.FERRAIL.REFONT.,414,4666,loading
11,CEREAL,E.MAIS VRAC,449,3810,loading
...,...,...,...,...,...
436,GAZ.L,I.BUTADIENE,436,2100,unloading
437,LIQ.V,I.HUILE COLZA,436,2997,unloading
439,SOL.V,I.SEL,433,3122,unloading
440,LIQ.V,I.EMHV - EMAG - FAME,436,1998,unloading


11

handling_type  handling_direction
AUTRES         loading                 2
               unloading              23
CEREAL         loading               102
GAZ.L          loading                 1
               unloading              27
LIQ.V          loading                49
               unloading              68
P.C.           loading                67
               unloading              66
SOL.V          loading                33
               unloading              72
Name: content_amount, dtype: int64

In [38]:
#EXTRACTION TUPLE 

contentType = pd.concat(
    [
        vesselCalls_df.loc[:, ["loading_cargo_fiscal_type", "loading_cargo_type"]], 
        vesselCalls_df.loc[:, ["unloading_cargo_fiscal_type", "unloading_cargo_type"]].rename(
            columns={
                'unloading_cargo_fiscal_type':'loading_cargo_fiscal_type',
                'unloading_cargo_type':'loading_cargo_type'
            }
        )
    ],
    axis= 0
)

contentType.columns = ["Group", "ID"]
contentType.replace({"":np.nan, 0:np.nan}, inplace=True)
contentType.dropna(axis= 0, subset= ["ID"], inplace=True)

In [40]:
contentType.ID.unique()

array(['E.HUILE TOURNESOL', 'E.MAIS VRAC', 'E.CONTENEURS',
       'E.FERRAIL.REFONT.', 'E.HUILES MIXTES', 'E.EMHV - EMAG - FAME ',
       'E.PNEUS BROYES', 'E.TRTX TOURNESOL', 'E.BLE VRAC',
       'E.COLIS LOURDS', 'E.GRAINES SOJA', 'E.PROD.CHIM.LIQ. AUTRES',
       'E.CEREALES MIXTES VRAC', 'E.SORGHO VRAC', 'E.TALC',
       'E.TERRE REFRACT.', 'E.COLZA VRAC', 'E.MINER.NON FERR.AUTRES',
       'E.CER.+OLEAG. MIXTES', 'E.HUILE COLZA', 'E.TRTX COLZA',
       'I.CHARBON', 'I.EMHV - EMAG - FAME', 'I.CLINKER',
       'I.TOURNESOL VRAC', 'I.CONTENEURS', 'I.TALL-OIL',
       'I.BOIS SCIES DU NORD', 'I.UREE VRAC', 'I.BUTADIENE', 'I.METHANOL',
       'I.HUILES MIXTES', 'I.HUILE COLZA', 'I.ENGR.MANUF.VRAC',
       'I.COLIS LOURDS', 'I.SEL', 'I.ENGR.LIQUIDES', 'I.KAOLIN',
       'I.LAITIERS', 'I.MELASSES', 'I.POTASSE VRAC', 'I.CIMENT',
       'I.MINERAUX AUTRES', 'I.HUILE TOURNESOL', 'I.TOURBE',
       'I.HUILE SOJA', 'I.COLZA VRAC', 'I.BOIS+COPEAUX MIXTES'],
      dtype=object)

## Assignations

## Supplychains

## Resources

# SCRATCHPAD

In [195]:
SC1, SC2, SC3 = {"SC_ID": "SC_1", "suitable": "Oui"}, \
                {"SC_ID": "SC_2", "suitable": "Oui"}, \
                {"SC_ID": "SC_3", "suitable": "Non"}  \
        
CT1, CT2, CT3 = {"possibles_SC": []},                 \
                {"possibles_SC": [SC1, SC2]},         \
                {"possibles_SC": [SC2, SC3]}          \

matching_CT = [CT1, CT2, CT3]

In [201]:
[SC for CT["possibles_SC"] in matching_CT for SC in CT["possibles_SC"] ]#if SC["suitable"]=="Oui" ]

['possibles_SC', 'possibles_SC', 'possibles_SC']

In [208]:
suitable_SC_list = []

for CT in matching_CT :
    if len(CT)>0:
        for SC in CT["possibles_SC"] :
            if SC["suitable"]=="Oui":
                suitable_SC_list.append(SC["SC_ID"])
            
set(suitable_SC_list)

TypeError: string indices must be integers

In [204]:
print(CT3)
print(CT3["possibles_SC"])

{'possibles_SC': {...}}
{'possibles_SC': {...}}
